# Gather protein structures and amino acid features
* This notebook generates a dict with diverse protein features for each protein
* Requires data from notebook #1

 **Include sequence alignments to the analysis**
* MSAs were generated using PSI-BLAST using the non-redudant protein sequences and limit the msas to 5000 sequences
* Aim: Test whether sequence conservation correlates with insertion tolerance
* The sequences were fetched via psi-blast with a limit of 5000 sequence
* MSAs were generated using MUSCLE with standard params
    * for large alignments the Super5 algorithm was used instead of PPP
* Kullback-Leibler-Divergence is taken as a measure for conservation

In [1]:
import pickle
import pandas as pd
from Bio import SeqIO
from Bio.PDB import *
import matplotlib.pyplot as plt
import matplotlib as matplotlib

from utils.plotting import *
from utils.processing import *

# set styles
plt.style.use('./utils/domain_ins.mplstyle')
plt.rcParams['svg.fonttype'] = 'none'

/camp/home/acarb/.conda/envs/DIscreen_env/lib/python3.7/site-packages/Bio/SubsMat/__init__.py:131: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  BiopythonDeprecationWarning,


### Define parameters

In [2]:
base = '/camp/lab/debenedictise/home/users/acarb/gitfolder/DI_screen_MathonyDebug'
in_folder = f'{base}/analysis/input_data'
data_folder = f'{base}/analysis/output_datasets'
fig_folder = f'{base}/analysis/figures'
primary_proteins = {rec.id : rec.seq for rec in SeqIO.parse(f'{in_folder}/proteins.fasta', 'fasta')}

#sequencing data:
# import analysis dict
with open(f'{data_folder}/analysis_dict.pickle', 'rb') as input:
    analysis_dict = pickle.load(input)
input.close()

# background AA frequences Swissprot release 2022_04
background_AA ={'A':0.0825, 'Q':0.0393, 'L':0.0965, 'S':0.0664, 'R':0.0553, 'E':0.0672, 'K':0.0580, 'T':0.0535, 'N':0.0406, 'G':0.0707,
                'M':0.0241, 'W':0.0110, 'D':0.0546, 'H':0.0227, 'F':0.0386, 'Y':0.0292, 'C':0.0138, 'I':0.0591, 'P':0.0474, 'V':0.0686}

#define features that should be taken into account 
feature_dict_radius = {'Hydrophobicity':'PRAM900101', 'Flexibility_idx':'BHAR880101', 'Molecular_weight':'FASG760101', 'Average_volume':'PONJ960101', 
'Positive_charge':'FAUJ880111', 'Negative_charge':'FAUJ880112', 'Net_charge':'KLEP840101', 'Radius_of_gyration':'LEVM760105',
 'Side-chain_stab_idx':'TAKK010101', 'Stability_scale_atom':'ZHOH040101', 'Stability_scale_experiment':'ZHOH040102', 'Buriability':'ZHOH040103'}
feature_dict_sequential = {'Linker_idx_Suyama':'SUYM030101', 'Linker_idx_George':'GEOR030101', 'Linker_idx_Bae':'BAEK050101'}

### Fetch features from AAindex DB, MSAs and PDB files

In [3]:
feat_dict_radius = import_features(feature_dict_radius)
feat_dict_sequential = import_features(feature_dict_sequential)

In [4]:
#process alignments and store data in dict
data_dict = {}
uniprot = {'AraC', 'TVMV', 'Flp', 'SigF'}
for protein in uniprot:
        data_dict[f'{protein}_data']={}
        data_dict[f'{protein}_data']['alignment_df'], data_dict[f'{protein}_data'][f'{protein}_positions'],  data_dict[f'{protein}_data']['alignment_counts'] = process_alignment(base, protein)
        data_dict[f'{protein}_data']['KLD'], data_dict[f'{protein}_data']['query_idx'] = KLD(data_dict[f'{protein}_data']['alignment_counts'], background_AA, data_dict[f'{protein}_data'][f'{protein}_positions'])
        data_dict[f'{protein}_data'] = insertion_stats(data_dict[f'{protein}_data']['query_idx'], primary_proteins[protein], data_dict[f'{protein}_data'])
        del data_dict[f'{protein}_data']['alignment_df'], data_dict[f'{protein}_data'][f'{protein}_positions'], data_dict[f'{protein}_data']['alignment_counts'], data_dict[f'{protein}_data']['query_idx']
        data_dict[f'{protein}_data'] = pd.DataFrame.from_dict(data_dict[f'{protein}_data'])


In [5]:
import os

# Path to the directory you want to add (e.g., the directory containing mkdssp)
new_path = os.path.expanduser('~/.conda/envs/DIscreen_env/bin')

# Get the current PATH
current_path = os.environ.get('PATH', '')

# Check if the new path is already in the PATH
if new_path not in current_path.split(os.pathsep):
    # Add the new path to the PATH
    os.environ['PATH'] = new_path + os.pathsep + current_path
    print(f"Added {new_path} to PATH")
else:
    print(f"{new_path} is already in PATH")

# Check the updated PATH
print("Current PATH:", os.environ['PATH'])

name='AraC'
base = base = '/camp/lab/debenedictise/home/users/acarb/gitfolder/DI_screen_MathonyDebug'
in_folder = f'{base}/analysis/input_data'
parser = PDBParser()
protein = parser.get_structure(name, f'{in_folder}/{name}_AF.pdb')
dssp = DSSP(protein[0], f'{in_folder}/{name}_AF.pdb', dssp='mkdssp')
sequence = ''
Sec_structure = ''
for z in range(len(dssp)):
    a_key = list(dssp.keys())[z]
    sequence += dssp[a_key][1]
    Sec_structure += dssp[a_key][2]
Sec_structure = Sec_structure.replace('-', 'C')
Sec_structure = Sec_structure.replace('I', 'C')
Sec_structure = Sec_structure.replace('T', 'C')
Sec_structure = Sec_structure.replace('S', 'C')
Sec_structure = Sec_structure.replace('G', 'H')
Sec_structure = Sec_structure.replace('E', 'B')
Sec_structure = [AA for AA in Sec_structure]


Added /camp/home/acarb/.conda/envs/DIscreen_env/bin to PATH
Current PATH: /camp/home/acarb/.conda/envs/DIscreen_env/bin:/camp/apps/eb/dev/software/JupyterLab/1.2.5-foss-2019b-Python-3.7.4/bin:/camp/apps/eb/dev/software/IPython/7.9.0-foss-2019b-Python-3.7.4/bin:/camp/apps/eb/dev/software/Tk/8.6.9-GCCcore-8.3.0/bin:/camp/apps/eb/dev/software/X11/20190717-GCCcore-8.3.0/bin:/camp/apps/eb/dev/software/fontconfig/2.13.1-GCCcore-8.3.0/bin:/camp/apps/eb/dev/software/expat/2.2.7-GCCcore-8.3.0/bin:/camp/apps/eb/dev/software/freetype/2.10.1-GCCcore-8.3.0/bin:/camp/apps/eb/dev/software/libpng/1.6.37-GCCcore-8.3.0/bin:/camp/apps/eb/software/SciPy-bundle/2019.10-foss-2019b-Python-3.7.4/bin:/camp/apps/eb/dev/software/ZeroMQ/4.3.2-GCCcore-8.3.0/bin:/camp/apps/eb/dev/software/util-linux/2.34-GCCcore-8.3.0/sbin:/camp/apps/eb/dev/software/util-linux/2.34-GCCcore-8.3.0/bin:/camp/apps/eb/dev/software/Python/3.7.4-GCCcore-8.3.0/bin:/camp/apps/eb/dev/software/SQLite/3.29.0-GCCcore-8.3.0/bin:/camp/apps/eb/dev

/camp/home/acarb/.conda/envs/DIscreen_env/lib/python3.7/site-packages/Bio/PDB/PDBParser.py:399: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2363
  PDBConstructionWarning,


In [6]:
#construct dataset
full_dataset = {}
input_pdbs = ['AraC', 'Flp', 'SigF','TVMV']

for protein in input_pdbs:  
    full_dataset[f'{protein}'] = {}
    full_dataset[f'{protein}']['2'] = collect_training_data(2, data_dict[f'{protein}_data'], protein, feat_dict_radius, feat_dict_sequential, primary_proteins, analysis_dict, in_folder)

#dump the current dataset to pickle
with open(f'{data_folder}/proteins_training.pickle', 'wb') as f:
    pickle.dump(full_dataset, f) 
f.close() 

/camp/home/acarb/.conda/envs/DIscreen_env/lib/python3.7/site-packages/Bio/PDB/PDBParser.py:399: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 2363
  PDBConstructionWarning,


In [7]:
for name, radius_dict in full_dataset.items():
    for name, df in radius_dict.items():
        df.dropna(inplace=True)

property_dict = {}
for i in analysis_dict.items():
        #select samples from second enrichment and average them, if 2 replicates are available
        temp_dict = {}
        for idx, frame in i[1].items():
            if len(idx) == 2 and idx[1] == '2':
                temp_dict[idx] = frame
        if len(temp_dict) == 1: 
            out_df = list(temp_dict.values())[0]['log']
        elif len(temp_dict) == 2:
            out_df = (temp_dict['12']['log'] + temp_dict['22']['log'])/2
        else:
            continue
        out_df = out_df.to_frame()
        complete_df = pd.concat([out_df.reset_index(), full_dataset[f'{i[0].split("_")[0]}']['2'].reset_index()], axis=1)
        complete_df.dropna(inplace=True)
        plot_df = complete_df[['log', 'Sec_structure', 'ASA', 'pLDDT']]   
        plot_df = plot_df.iloc[1:-1,:]
        plot_df = pd.melt(plot_df, id_vars=['Sec_structure', 'ASA', 'pLDDT'], value_vars=out_df.columns, value_name='enrichment')
        plot_df['ASA'] =plot_df['ASA'].astype('float')
        plot_df['pLDDT'] =plot_df['pLDDT'].astype('float')
        property_dict[i[0]] = complete_df
        feature_correlation_plot(plot_df, i[0], 'ASA', fig_folder)
        feature_correlation_plot(plot_df, i[0], 'pLDDT', fig_folder)
        violin_plot(plot_df, i[0], 'Sec_structure', fig_folder)


findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
/nemo/lab/debenedictise/home/users/acarb/gitfolder/DI_screen_MathonyDebug/analysis/utils/plotting.py:122: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(5,5))
/nemo/lab/debenedictise/home/users/acarb/gitfolder/DI_screen_MathonyDebug/analysis/utils/plotting.py:105: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(5,5))
/nemo/lab/debenedictise/home/users/acarb